In [196]:
import json
import openai

In [197]:
client = openai.Client()

In [198]:
def saudacao_por_periodo(hora):
    if hora < 12:
        return json.dumps({ "saudacao": "Bom dia!" })
    elif hora < 18:
        return json.dumps({ "saudacao": "Boa tarde!" })
    else:
        return json.dumps({ "saudacao": "Boa noite!" })

def ask():
    return json.dumps({ "answer": "Não entendi a pergunta." })



In [199]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "saudacao_por_periodo",
            "description": "Retorna uma saudação de acordo com o período do dia",
            "parameters": {
                "type": "object",
                "properties": {
                    "hora": { 
                        "type": "integer", 
                        "description": "Hora do dia em formato de 24h" 
                    }
                },
                "required": ["hora"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "ask",
            "description": "Quando não entende a pergunta",
        }
    }
]

In [200]:
funcao_disponivel = {
    "saudacao_por_periodo": saudacao_por_periodo,
    "ask": ask
}

In [201]:
mensagens = [{ "role": "user", "content": "Olá, quero um hamburguer" }]

In [202]:
resposta = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=mensagens,
    tools=tools,
    tool_choice="auto"
)

In [203]:
resp = resposta.choices[0].message
resp

ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_yqLzTfIxQWTlx86hDr6iaZyv', function=Function(arguments='{}', name='ask'), type='function')])

In [204]:
tools_calls = resp.tool_calls
tools_calls

[ChatCompletionMessageToolCall(id='call_yqLzTfIxQWTlx86hDr6iaZyv', function=Function(arguments='{}', name='ask'), type='function')]

In [ ]:
if tools_calls:
    mensagens.append(resp)
    for tool_call in tools_calls:
        function_name = tool_call.function.name
        function_to_call = funcao_disponivel[function_name]
        function_args = json.loads(tool_call.function.arguments)
        function_result = function_to_call(
            **function_args
        )
        mensagens.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_result
            }
        )
    
    segunda_resposta = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=mensagens,
        tools=tools,
        tool_choice="auto"
    )

In [206]:
mensagem_resposta = segunda_resposta.choices[0].message
mensagem_resposta

ChatCompletionMessage(content='Desculpe, eu não entendi o que você quer. Como posso te ajudar hoje?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)